In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import torch
from torch.nn import functional as F

from config import *

from src.datafeed.downstream import get_fx_data

In [23]:
# helper to print dataframes nicely
def style_time_series(df, n_tail=6, mult=1.0, precision=2):
    _res = df.tail(n_tail).mul(mult).rename_axis(index="date")
    _res.index = _res.index.strftime("%Y-%m-%d")
    _res = _res.style.format(precision=precision)
    return _res

In [15]:
# load data
data = get_fx_data()

For each currency, I calculate 1-month excess returns $rx$ in a forward-looking manner, that is, $rx_t = \frac{S_{t+1m}}{F_t}$. Note that the returns are observed daily, which increases the number of data points twenty-fold compared to the case when monthly returns are observed monthly.

In [24]:
rx = data["excess_returns"]

print("1-month forward-looking excess returns, in %")
display(style_time_series(rx.loc[:"2020-12-01"], n_tail=3, mult=100))

1-month forward-looking excess returns, in %


currency,aud,cad,chf,dkk,eur,gbp,jpy,nok,nzd,sek
date,,,,,,,,,,
2020-11-27,3.73,2.06,2.67,2.32,2.27,1.90,0.85,3.25,3.20,2.87
2020-11-30,4.33,1.72,2.96,2.64,2.57,1.80,1.06,3.75,3.34,3.77
2020-12-01,3.95,1.21,1.92,1.41,1.35,1.06,1.08,2.88,2.65,2.62


For each currency, define the date-$t$ term structure history (TSH) as the $T \times M$ matrix containing the history of forward prices for $M$ tenors over the past $T$ periods.

In [25]:
# display term struct history
tsh = data["term_structure_history"]

print("6-day term structure history of aud on 2020-12-01")
display(
    style_time_series(
        tsh.loc[:"2020-12-01", "aud"], 
        precision=4
    ).background_gradient(cmap='Reds')
)

6-day term structure history of aud on 2020-12-01


maturity,spot,1m,2m,3m,6m,9m,12m,2y
date,,,,,,,,
2020-11-24,0.7361,0.7363,0.7365,0.7366,0.7369,0.7372,0.7375,0.7384
2020-11-25,0.7365,0.7367,0.7368,0.7370,0.7373,0.7376,0.7379,0.7387
2020-11-26,0.7362,0.7364,0.7365,0.7367,0.7370,0.7373,0.7376,0.7384
2020-11-27,0.7387,0.7389,0.7391,0.7392,0.7395,0.7398,0.7401,0.7408
2020-11-30,0.7344,0.7347,0.7348,0.7350,0.7353,0.7356,0.7358,0.7363
2020-12-01,0.7371,0.7374,0.7376,0.7377,0.7381,0.7384,0.7386,0.7392


Let's also define the normalized term structure history NTSH as follows: divide all values by the date-$t$ spot price, take the log and 'annualize' all values except the spot column:

In [7]:
annualizer = pd.Series({"spot": 1,
                        "1m"  : 12/1,
                        "2m"  : 12/2,
                        "3m"  : 12/3,
                        "6m"  : 12/6,
                        "9m"  : 12/9,
                        "12m" : 12/12,
                        "2y"  : 12/24})

def normalize_term_structure(ts):
    _res = ts\
        .div(ts.iloc[-1].xs("spot", level="maturity"), axis=1, level="currency")\
        .pipe(np.log)\
        .mul(annualizer, axis=1, level="maturity")
    return _res

In [26]:
print("normalized 6-day term structure history of aud on 2020-12-01")
display(
    style_time_series(
        normalize_term_structure(
            data["term_structure_history"].loc[:"2020-12-01", ["aud"]].tail(6)
        ), 
        precision=4
    ).background_gradient(cmap='Reds')
)

normalized 6-day term structure history of aud on 2020-12-01


This normalized matrix encodes many signals commonly used to construct FX strategies. For instance, a carry trade signal can be extracted by simply observing the most recent value in column '1m', the 5-day momentum signal &ndash; by observing the value in column 'spot' five days from today, a skewness signal &ndash; by subtracting the most recent value in column '12m' from that in columne '1m', and so on. In fact, any signal that is a linear transformation of the NTSH can be extracted with a suitable convolution. For instance, the following convolution extracts the 1-month carry signal:

In [27]:
ntsh = normalize_term_structure(
    data["term_structure_history"].loc[:"2020-12-01", ["aud"]].tail(6)
)

# to tensor
ntsh_t = torch.from_numpy(ntsh.values)

# convolution for carry signal
carry_conv = torch.zeros_like(ntsh_t, dtype=float)
carry_conv[-1, 1] = 1

print("NTSH:")
display(style_time_series(ntsh, precision=4))

print("carry convolution:")
print(carry_conv)

print("\nextracting the 1-month carry signal:")
print(
    F.conv2d(ntsh_t[None, None, ...], 
             carry_conv[None, None, ...]).squeeze()
)

NTSH:


carry convolution:
tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.]], dtype=torch.float64)

extracting the 1-month carry signal:
tensor(0.0046, dtype=torch.float64)


Applied to the whole cross-section of currencies:

In [30]:
ntsh = normalize_term_structure(
    data["term_structure_history"].loc[:"2020-12-01"].tail(6)
)

# to tensor
ntsh_t = torch.from_numpy(ntsh.values)
n_tenors = len(ntsh.columns.unique("maturity"))

print("extracting the 1-month carry signal:")
print(
    F.conv2d(ntsh_t[None, None, ...], 
             carry_conv[None, None, ...],
             stride=(1, n_tenors)).squeeze()
)

print("1-month forward discounts:")
display(style_time_series(ntsh.xs("1m", axis=1, level="maturity").tail(1),
                          precision=4))



extracting the 1-month carry signal:
tensor([0.0046, 0.0032, 0.0165, 0.0129, 0.0135, 0.0073, 0.0101, 0.0031, 0.0011,
        0.0110], dtype=torch.float64)
1-month forward discounts:


currency,aud,cad,chf,dkk,eur,gbp,jpy,nok,nzd,sek
date,,,,,,,,,,
2020-12-01,0.0046,0.0032,0.0165,0.0129,0.0135,0.0073,0.0101,0.0031,0.0011,0.0110


These can be used with the softmax activation function to get the weights in a long-short portfolio. To do so, I normalize the signals by subtracting the mean and dividing by the standard deviation, and take the modified softmax, forcing the values to be approximately between -1 and 1 and sum to zero.

In [60]:
# calculate signals
signals = F.conv2d(ntsh_t[None, None, ...], 
                   carry_conv[None, None, ...],
                   stride=(1, n_tenors)).squeeze()
n_assets = len(signals)

# normalize, take softmax with temperature parameter = 1.5
w = (F.softmax((signals - torch.mean(signals)) / torch.std(signals) * 1.5, dim=0) - 1/n_assets) * 2

print("long-short portfolio weights:")
print(w)

long-short portfolio weights:
tensor([-0.1700, -0.1800,  0.6700,  0.1000,  0.1700, -0.1400, -0.0600, -0.1800,
        -0.1900, -0.0200], dtype=torch.float64)


These weights are different from the ones frequently used in long-short rank-based strategies, but, hopefully, close enough. The carry trade strategy constructed with such weights is profitable, as shown below:

In [61]:
# normalize and take softmax with a temperature parameter
def normalized_softmax(_x, dim=-1, temp=1.5):
    _res = (
        F.softmax(
            (_x - torch.mean(_x, dim=dim, keepdim=True)) / \
                torch.std(_x, dim=dim, keepdim=True) * \
                temp, 
            dim=dim
        ) - 1/n_assets
    ) * 2
    return _res

In [64]:
_fd = data["fd_annualized"].xs("1m", axis=1, level="maturity")
fd = torch.from_numpy(_fd.values)
signals_carry = pd.DataFrame(normalized_softmax(fd).numpy(), 
                             index=_fd.index, columns=_fd.columns)
carry_rx = rx.mul(signals_carry).sum(axis=1)

print("avg return of the modified carry trade strategy, annualized:")
print(np.round(carry_rx.mean() * 1200, 2))

avg return of the modified carry trade strategy, annualized:
3.95
